## Make sure to run Chunking Series and Durations first. This will not be accurate if those files are not up to date.

In [1]:
import pandas as pd
import numpy as np

In [6]:
scaleColumns = ['EntryFeeAmount', 'TotalPrizeAmount','MaxNumberPlayers', 'MaxEntriesPerUser', 'DistinctUsers', 'NumGames', 'DraftablePlayersInSet','PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition', 'Duration']
categoricalColumns = ['SportName', 'VariantName', 'Contest_Group']

In [80]:
rawDF = pd.read_csv('data/WorkingData.csv')
rawDF = pd.merge(rawDF, pd.read_csv('data/Durations.csv').set_index('ContestId'), on='ContestId', how='left')

In [81]:
abDF = pd.read_csv('data/AB_Preds.csv').set_index('ContestId').drop(columns='Duration')
rawDF = pd.merge(rawDF, abDF, on='ContestId', how='left').set_index('ContestId')

In [82]:
processedDF = rawDF.copy(deep=True)


In [84]:
processedDF = rawDF.copy(deep=True)
metaColumns = []
abColumns = []
successCol = []


processedDF['Success'] = (processedDF['MaxNumberPlayers'] == processedDF['Entries'])
successCol.append('Success')

successDF = processedDF[['Success']]

for scol in scaleColumns:
    processedDF[scol+"_Scaled"] = processedDF[scol]/processedDF[scol].max()
    metaColumns.append(scol+"_Scaled")
    
for scol in categoricalColumns:
    for val in processedDF[scol].unique():
        catStr = scol+"_"+val
        processedDF[catStr] = (processedDF[scol]==val)
        metaColumns.append(catStr)
        
metaDF = processedDF[metaColumns]

for col in abDF.columns:
    if(col[:4]=="Pred"):
        label = col+"_Scaled"
        processedDF[label] = processedDF[col]/processedDF["MaxNumberPlayers"]
        abColumns.append(label)
        
kalmanPredsDF = processedDF[abColumns]

In [65]:
kalmanPredsDF.to_csv('data/KalmanResults.csv')
metaDF.to_csv('data/MetaData.csv')

In [86]:
workedData = pd.merge(metaDF, kalmanPredsDF, on='ContestId', how='left')
workedData = pd.merge(successDF, workedData, on='ContestId', how='left')

In [89]:
workedData.to_csv('data/WorkedData.csv')